<a href="https://colab.research.google.com/github/donghui-0126/mini-project/blob/main/shoes-project/resell_regressor/bn_dnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from sklearn.linear_model import LinearRegression
import warnings
from sklearn.decomposition import PCA
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import tensorflow as tf
from tensorflow.keras.layers import Dense, Dropout, Embedding, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras import Input



warnings.filterwarnings('ignore')

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
df = pd.read_csv('/content/drive/MyDrive/kream_data/product_data_dpp.csv')

In [4]:
df.shape

(6103, 15)

In [5]:
df_nike = df[df['brand']=='Nike'][['brand', 'name', 'price_og', 'price_resell']]
df_jordan = df[df['brand']=='Jordan'][['brand', 'name', 'price_og', 'price_resell']]
df_adidas = df[df['brand']=='Adidas'][['brand', 'name', 'price_og', 'price_resell']]

In [6]:
print(df_nike.shape)
df_nike.head()

(1455, 4)


,brand,name,price_og,price_resell
0,Nike,Nike Dunk Low Retro Black,129000,152800
1,Nike,Nike Air Force 1 '07 Low White,139000,136200
2,Nike,Nike Air Force 1 '07 WB Flax,169000,173200
3,Nike,Nike x Supreme Air Force 1 Low Flax,184600,281400
4,Nike,Nike Zoom Vomero 5 PRM Light Iron Ore and Flat...,209000,258000


In [7]:
print(df_jordan.shape)
df_jordan.head()

(1376, 4)


,brand,name,price_og,price_resell
1455,Jordan,Jordan 1 Retro High OG Chicago 2022,209000,480800
1456,Jordan,Jordan 1 Low Concord,139000,163600
1457,Jordan,Jordan 1 High Golf White Black,249000,254400
1458,Jordan,Jordan 1 x Travis Scott Retro Low OG SP Black ...,189000,698000
1459,Jordan,Jordan 1 x Travis Scott Retro Low OG SP Sail a...,189000,1417800


In [42]:
df_jordan.loc[1459, 'input']

'Jordan | Jordan 1 x Travis Scott Retro Low OG SP Sail and Ridgerock | 18.9'

In [12]:
df_nike = df_nike.drop(df_nike[df_nike['price_resell']>8000000].index)

In [13]:
df_nike['price_class'] = np.where(df_nike['price_resell'] <1000000,\
                                  df_nike['price_resell']//100000,\
                                  10)

-----

In [14]:
df_jordan = df_jordan.drop(df_nike[df_nike['price_resell']>5000000].index)

In [15]:
df_jordan['price_class'] = np.where(df_jordan['price_resell'] <500000,\
                                  df_jordan['price_resell']//200000,\
                                  13)

---------

In [16]:
def make_input_col(df_list):
    for df in df_list:
        df['input'] = df['brand'] + " | " + df['name'] + " | " + (df['price_og']/10000.0).astype(str)
        df['target'] = df['price_resell']/10000.0

make_input_col([df_nike, df_jordan, df_adidas])

In [49]:
def tokenize_input(df_list, pd_list):
    return_list = []

    for df, pd in zip(df_list, pd_list):
        tokenizer = Tokenizer()
        tokenizer.fit_on_texts(df['input'])
        word_index = tokenizer.word_index

        print(f"word_index의 크기: {len(word_index)}")

        # 문장을 토큰화 한다.
        tokenized_name = tokenizer.texts_to_sequences(df['input'])
        padded_name = pad_sequences(tokenized_name)
        return_list.append((padded_name))
    return return_list

df_nike_input_ = pd.DataFrame()
df_jordan_input_ = pd.DataFrame()
df_adidas_input_ = pd.DataFrame()

[df_nike_input, df_jordan_input, df_adidas_input] = tokenize_input([df_nike, df_jordan, df_adidas], [df_nike_input_, df_jordan_input_, df_adidas_input_])

word_index의 크기: 1215
word_index의 크기: 967
word_index의 크기: 1046


In [18]:
# # tokenizer 저장
# # python 변수(객체)를 pickle 파일로 저장
# with open('bn_tokenizer.pickle', 'wb') as f:
#     pickle.dump(tokenizer, f)

# # tokenizer load 하기
# with open('bn_tokenizer.pickle', 'rb') as f:
#     tokenizer = pickle.load(f)

In [19]:
from sklearn.model_selection import train_test_split

In [20]:
nike_train_X, nike_test_X, nike_train_y, nike_test_y = train_test_split(df_nike_input, df_nike[['target', 'price_class']], test_size=0.3, shuffle=True, stratify=df_nike['price_class'])
nike_valid_X, nike_test_X, nike_valid_y, nike_test_y = train_test_split(nike_test_X, nike_test_y, test_size=0.5)

jordan_train_X, jordan_test_X, jordan_train_y, jordan_test_y = train_test_split(df_jordan_input, df_jordan[['target', 'price_class']], test_size=0.3, shuffle=True, stratify=df_jordan['price_class'])
jordan_valid_X, jordan_test_X, jordan_valid_y, jordan_test_y = train_test_split(jordan_test_X, jordan_test_y, test_size=0.5)

In [21]:
%cd /content/drive/MyDrive/kream_data/model

/content/drive/MyDrive/kream_data/model


## 클래스, 회귀값 동시 에측

In [24]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

es=EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=30)
mc_jordan2=ModelCheckpoint('nike_bn_best_model_2.h5', monitor='val_loss', verbose=1, save_best_only=True)
mc_jordan3=ModelCheckpoint('jordan_bn_best_model_3.h5', monitor='val_loss', verbose=1, save_best_only=True)

In [25]:
def focal_loss(gamma=2.0, alpha=0.25):
    def focal_loss_fn(y_true, y_pred):
        y_true = tf.cast(y_true, tf.float32)
        y_pred = tf.cast(y_pred, tf.float32)


        y_pred = tf.clip_by_value(y_pred, 1e-7, 1.0 - 1e-7)  # 예측 값 클리핑
        pt = y_true * y_pred + (1 - y_true) * (1 - y_pred)  # 실제 클래스와의 일치 여부
        focal_loss = -alpha * tf.math.pow(1.0 - pt, gamma) * tf.math.log(pt)  # Focal Loss 계산
        return tf.reduce_sum(focal_loss)  # 평균 대신 합계를 계산
    return focal_loss_fn

In [33]:
# 입력 데이터 형태를 정의합니다.
input_shape = (16,)  # 입력 데이터의 형태에 맞게 설정합니다.

# 클래스 개수 정의
num_classes = 13
word_size=500
# 입력 레이어를 정의합니다.
inputs = Input(shape=input_shape)

# 임베딩층을 추가합니다.
embedding = Embedding(word_size, word_size, input_length=16)(inputs)
flatten = Flatten()(embedding)

# 중간 레이어를 구성합니다.
hidden1 = Dense(64, activation='relu')(flatten)
hidden2 = Dense(32, activation='relu')(hidden1)
hidden3 = Dense(8, activation='relu')(hidden2)


# 첫 번째 출력을 위한 레이어를 정의합니다.
output1 = Dense(1, activation='linear', name='output1')(hidden3)  # 특정 값에 대한 출력

# 두 번째 출력을 위한 레이어를 정의합니다.
output2 = Dense(num_classes, activation='softmax', name='output2')(hidden2)  # 클래스에 대한 출력

# 다중 출력 모델을 생성합니다.
nike_bn_model = Model(inputs=inputs, outputs=[output1, output2])

# 모델을 컴파일합니다.
nike_bn_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
                      loss={'output1': 'mse', 'output2': 'sparse_categorical_crossentropy'},
                      metrics={'output1': 'mae', 'output2': 'accuracy'})

# 모델을 학습합니다.
nike_bn_model.fit(jordan_train_X, [jordan_train_y['target'], jordan_train_y['price_class']], batch_size=64, epochs=10,
                  validation_data=(jordan_valid_X, [jordan_valid_y['target'], jordan_valid_y['price_class']]),
                  callbacks=[mc_jordan2])


Epoch 1/10
16/16 [==============================] - ETA: 0s - loss: nan - output1_loss: nan - output2_loss: nan - output1_mae: nan - output2_accuracy: 0.3385
Epoch 1: val_loss did not improve from inf
16/16 [==============================] - 4s 119ms/step - loss: nan - output1_loss: nan - output2_loss: nan - output1_mae: nan - output2_accuracy: 0.3385 - val_loss: nan - val_output1_loss: nan - val_output2_loss: nan - val_output1_mae: nan - val_output2_accuracy: 0.3447
Epoch 2/10
16/16 [==============================] - ETA: 0s - loss: nan - output1_loss: nan - output2_loss: nan - output1_mae: nan - output2_accuracy: 0.3479
Epoch 2: val_loss did not improve from inf
16/16 [==============================] - 5s 320ms/step - loss: nan - output1_loss: nan - output2_loss: nan - output1_mae: nan - output2_accuracy: 0.3479 - val_loss: nan - val_output1_loss: nan - val_output2_loss: nan - val_output1_mae: nan - val_output2_accuracy: 0.3447
Epoch 3/10
15/16 [===========================>..] - ETA:

In [32]:
# 입력 데이터 형태를 정의합니다.
input_shape = (16,)  # 입력 데이터의 형태에 맞게 설정합니다.

# 클래스 개수 정의
num_classes = 13
word_size=500
# 입력 레이어를 정의합니다.
inputs = Input(shape=input_shape)

# 임베딩층을 추가합니다.
embedding = Embedding(word_size, word_size, input_length=16)(inputs)
flatten = Flatten()(embedding)

# 중간 레이어를 구성합니다.
hidden1 = Dense(64, activation='relu')(flatten)
hidden2 = Dense(32, activation='relu')(hidden1)
hidden3 = Dense(8, activation='relu')(hidden2)


# 첫 번째 출력을 위한 레이어를 정의합니다.
output1 = Dense(1, activation='linear', name='output1')(hidden3)  # 특정 값에 대한 출력

# 두 번째 출력을 위한 레이어를 정의합니다.
output2 = Dense(num_classes, activation='softmax', name='output2')(hidden2)  # 클래스에 대한 출력

# 다중 출력 모델을 생성합니다.
nike_bn_model = Model(inputs=inputs, outputs=[output1, output2])

# 모델을 컴파일합니다.
nike_bn_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
                      loss={'output1': 'mse', 'output2': focal_loss(gamma=2.0, alpha=0.25)},
                       metrics={'output1': 'mae', 'output2': 'accuracy'})

# 모델을 학습합니다.
nike_bn_model.fit(jordan_train_X, [jordan_train_y['target'], jordan_train_y['price_class']], batch_size=64, epochs=10,
                  validation_data=(jordan_valid_X, [jordan_valid_y['target'], jordan_valid_y['price_class']]),
                  callbacks=[mc_jordan3])


Epoch 1/10
16/16 [==============================] - ETA: 0s - loss: nan - output1_loss: nan - output2_loss: nan - output1_mae: nan - output2_accuracy: 0.3240
Epoch 1: val_loss did not improve from inf
16/16 [==============================] - 5s 168ms/step - loss: nan - output1_loss: nan - output2_loss: nan - output1_mae: nan - output2_accuracy: 0.3240 - val_loss: nan - val_output1_loss: nan - val_output2_loss: nan - val_output1_mae: nan - val_output2_accuracy: 0.3447
Epoch 2/10
16/16 [==============================] - ETA: 0s - loss: nan - output1_loss: nan - output2_loss: nan - output1_mae: nan - output2_accuracy: 0.3479
Epoch 2: val_loss did not improve from inf
16/16 [==============================] - 3s 159ms/step - loss: nan - output1_loss: nan - output2_loss: nan - output1_mae: nan - output2_accuracy: 0.3479 - val_loss: nan - val_output1_loss: nan - val_output2_loss: nan - val_output1_mae: nan - val_output2_accuracy: 0.3447
Epoch 3/10
14/16 [=========================>....] - ETA:

In [ ]:
from sklearn.metrics import mean_squared_error

In [73]:
import lightgbm as lgbm
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score, roc_auc_score


lgbm_model = lgbm.LGBMRegressor(n_estimators=1000, num_leaves=64, n_jobs=-1, boost_from_average=True)

# 모델 학습
lgbm_model.fit(jordan_train_X, jordan_train_y['price_class'])

model = lgbm.LGBMRegressor(random_state=32)
model.fit(
    jordan_train_X,
    jordan_train_y['target'],
    eval_set=[(jordan_train_X, jordan_train_y['target']),(jordan_valid_X, jordan_valid_y['target'])],
    eval_metric='rmse',
    verbose=0
    )

pred = model.predict(jordan_test_X)

mean_squared_error(pred, jordan_test_y['target'])**0.5

78.22185548153341

In [ ]:
import pickle

In [81]:
with open('tokenizer/bn_jordan_tokenizer.pickle', 'rb') as f:
    bn_tokenizer = pickle.load(f)

BN = 'Jordan | Jordan 1 CHICAGO TRAVIS'

bn_tokenized_name = bn_tokenizer.texts_to_sequences(pd.Series(BN))
bn_padded_name = pd.DataFrame(pad_sequences(bn_tokenized_name, maxlen=16))

bn_pred = pd.DataFrame(model.predict(bn_padded_name))

print(bn_pred[0])

0    85.272124
Name: 0, dtype: float64


In [58]:
bn_tokenized_name

[[1, 1, 3, 16, 134, 135, 2, 5, 18, 43, 66, 55, 537, 35, 4], [7, 11]]